# CIS 194: Homework 8

## *Preliminaries*

We have provided you with the file Employee.hs, which contains the
following definitions:

```haskell
-- Employee names are represented by Strings.
type Name = String

-- The amount of fun an employee would have at the party,
-- represented by an Integer number of STFUs
type Fun = Integer

-- An Employee consists of a name and a fun score.
data Employee = Emp {empName :: Name, empFun :: Fun}
  deriving (Show, Read, Eq)
```

It also defines `testCompany :: Tree Employee`, a small company
hierarchy which you can use for testing your code (although your
actual company hierarchy is much larger).

Finally, `Employee.hs` defines a type to represent guest lists. The
obvious possibility to represent a guest list would be `[Employee]`.
However, we will frequently want to know the total amount of fun
had by a particular guest list, and it would be inefficient to recompute it every time by adding up the fun scores for all the employees
in the list. Instead, a `GuestList` contains both a list of `Employees`
and a Fun score. Values of type GuestList should always satisfy the
invariant that the sum of all the Fun scores in the list of `Employees`
should be equal to the one, “cached” Fun score.

In [1]:
import Data.Ord
import Data.Tree
import Data.List
import Data.Bifunctor

:l Employee

## Exercise 1

Now define the following tools for working with `GuestLists`:

1. A function
```haskell
glCons :: Employee -> GuestList -> GuestList
```

which adds an `Employee` to the `GuestList` (updating the cached
Fun score appropriately). Of course, in general this is impossible:
the updated fun score should depend on whether the `Employee`
being added is already in the list, or if any of their direct subordinates are in the list, and so on. For our purposes, though, you
may assume that none of these special cases will hold: that is,
glCons should simply add the new `Employee` and add their fun
score without doing any kind of checks.

2. A `Monoid` instance for `GuestList`.(How is the Monoid instance supposed to work, you ask? You figure it out!)
3. A function `moreFun :: GuestList -> GuestList -> GuestList`
which takes two `GuestLists` and returns whichever one of them
is more fun, *i.e.* has the higher fun score. (If the scores are equal it
does not matter which is returned.)

In [2]:
glCons :: Employee -> GuestList -> GuestList
glCons e@(Emp _ f) (GL l tf) = GL (e : l) (f + tf)

instance Semigroup GuestList where
  (GL l lf) <> (GL r rf) = GL (l ++ r) (lf + rf)

instance Monoid GuestList where
  mempty = GL [] 0

moreFun :: GuestList -> GuestList -> GuestList
moreFun = max

## Exercise 2

The `Data.Tree` module from the standard Haskell libraries defines
the type of “rose trees”, where each node stores a data element and
has any number of children (*i.e.* a *list* of subtrees):

```haskell
data Tree a = Node {
        rootLabel :: a, -- label value
        subForest :: [Tree a] -- zero or more child trees
    }
```

Strangely, `Data.Tree` does *not* define a fold for this type! Rectify the
situation by implementing

```haskell
treeFold :: ... -> Tree a -> b
```

(See if you can figure out what type(s) should replace the dots in
the type of `treeFold`. If you are stuck, look back at the lecture notes
from Week 7, or infer the proper type(s) from the remainder of this
assignment.)

In [3]:
treeFold :: (a -> [b] -> b) -> Tree a -> b
treeFold f (Node v subtrees) = f v (map (treeFold f) subtrees)

In [4]:
r = treeFold f testCompany
  where
    f x ys = empFun x + sum ys
r

46

## *The algorithm*

Now let’s actually derive an algorithm to solve this problem. Clearly
there must be some sort of recursion involved—in fact, it seems that
we should be able to do it with a fold. This makes sense though
starting from the bottom of the tree and working our way up, we
compute the best guest list for each subtree and somehow combine
these to decide on the guest list for the next level up, and so on. So
we need to write a combining function

```haskell
combineGLs :: Employee -> [GuestList] -> GuestList
```

which takes an employee (the boss of some division) and the optimal
guest list for each subdivision under him, and somehow combines
this information to compute the best guest list for the entire division.

However, this obvious first attempt fails! The problem is that we
don’t get enough information from the recursive calls. If the best
guest list for some subtree involves inviting that subtree’s boss, then
we are stuck, since we might want to consider inviting the boss of the
entire tree—in which case we don’t want to invite any of the subtree
bosses (since they wouldn’t have any fun anyway). But we might be
able to do better than just taking the best possible guest list for each
subtree and then excluding their bosses.

The solution is to generalize the recursion to compute *more* information, in such a way that we can actually make the recursive step.
In particular, instead of just computing the best guest list for a given
tree, we will compute *two* guest lists:

1. the best possible guest list we can create *if we invite the boss* (that
is, the `Employee` at the root of the tree); and
2. the best possible guest list we can create if we *don’t* invite the boss.

It turns out that this gives us enough information at each step to
compute the optimal two guest lists for the next level up.

## Exercise 3

Write a function
```haskell
nextLevel :: Employee -> [(GuestList, GuestList)]
    -> (GuestList, GuestList)
```

which takes two arguments. The first is the “boss” of the current subtree (let’s call him Bob). The second argument is a list of the results
for each subtree under Bob. Each result is a pair of `GuestLists`: the
first GuestList in the pair is the best possible guest list *with* the boss
of that subtree; the second is the best possible guest list *without* the
boss of that subtree. nextLevel should then compute the overall best
guest list that includes Bob, and the overall best guest list that doesn’t
include Bob.

In [5]:
nextLevel ::
  Employee ->
  [(GuestList, GuestList)] ->
  (GuestList, GuestList)
nextLevel emp res = (withEmp, withoutEmp)
  where
    withEmp = glCons emp $ mconcat (map snd res)
    withoutEmp = mconcat $ map (uncurry max) res

## Exercise 4

Finally, put all of this together to define
```haskell
maxFun :: Tree Employee -> GuestList
```

which takes a company hierarchy as input and outputs a fun-maximizing
guest list. You can test your function on `testCompany`, provided in
`Employee.hs`.

In [6]:
maxFun :: Tree Employee -> GuestList
maxFun = uncurry moreFun . treeFold nextLevel

In [7]:
maxFunSt :: Tree Employee -> GuestList
maxFunSt = uncurry moreFun . foldTree nextLevel

assumeMax t = maxFun t == maxFunSt t

In [8]:
assumeMax testCompany

True

In [9]:
assumeMax testCompany2

True

In [10]:
readFile "company.txt" >>= (print . assumeMax . read)

True

## Exercise 5

Implement `main :: IO ()` so that it reads your company’s hierarchy from the file `company.txt`, and then prints out a formatted guest
list, sorted by first name, which looks like

```
Total fun: 23924
Adam Debergues
Adeline Anselme
...
```

(Note: the above is just an example of the format; it is not the correct
output!) You will probably find the readFile and putStrLn functions
useful.

As much as possible, try to separate out the “pure” computation
from the IO computation. In other words, your main function should
actually be fairly short, calling out to helper functions (whose types
do not involve IO) to do most of the work. If you find IO “infecting”
all your function types, you are Doing It Wrong.

In [11]:
format :: GuestList -> IO ()
format (GL lst fun) =
  putStrLn ("Total fun: " ++ show fun)
    >> (putStr . unlines . sort . map empName $ lst)

In [12]:
readFile "company.txt" >>= (format . maxFun . read)

Total fun: 33200
Adam Bohrmann
Adam Briscoe
Adam Debergues
Adam Donalon
Adam Eschmann
Adelaide Ahrens
Adelaide Eames
Adeline Anselme
Adeline Beyer
Adeline Cottem
Adrien Benjamin
Adrien Coll
Adrien Connell
Adrien Demuth
Adrien Fettig
Adrien Fostin
Agathe Brownlee
Agathe Cregg
Agathe Frilo
Albert Anselme
Albert Cappo
Alexis Cahill
Alexis Durandi
Alfred Barnes
Alfred Bern
Alfred Bowles
Alfred Conrad
Alfred Dannemann
Alice Betzer
Alice Boree
Aline Boulanger
Aline Deruisse
Allan Smith Anselmet
Allan Smith Bischoff
Allan Smith Frechett
Alonzo C. Bonaud
Amelia Blaetz
Amelia Burkenslock
Amelie Anselme
Amelie Flynn
Andre Aveigno
Andre Basart
Andre Burvant
Andre Corr
Angelo Angel
Angelo Carrico
Angelo Drayfus
Angelo Fanz
Anna Barbon
Anna Becker
Anna Bloch
Anna Laura Casler
Anna Maria Cohen
Anne Bardeau
Anne Conners
Anne Denis
Anne Donnahou
Annie Barrere
Annie Cambias
Annie Davis
Anthony Adams
Anthony Bacchus
Anthony Beddinghaus
Anthony Duffey
Arsene Borle
Arsene Cathalougne
Arsene Dromel
August 